In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")
    
    # Set up logging for predictions
    tensors_to_log = {}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001499FDBA8D0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model\model.ckpt.
INFO:ten

INFO:tensorflow: (26.545 sec)
INFO:tensorflow:global_step/sec: 1.75479
INFO:tensorflow: (30.375 sec)
INFO:tensorflow:loss = 0.307588, step = 4402 (56.921 sec)
INFO:tensorflow: (33.418 sec)
INFO:tensorflow:global_step/sec: 1.64237
INFO:tensorflow: (27.470 sec)
INFO:tensorflow:loss = 0.325458, step = 4502 (60.887 sec)
INFO:tensorflow: (26.670 sec)
INFO:tensorflow:global_step/sec: 1.74313
INFO:tensorflow: (30.759 sec)
INFO:tensorflow:loss = 0.295442, step = 4602 (57.429 sec)
INFO:tensorflow: (33.333 sec)
INFO:tensorflow:global_step/sec: 1.66134
INFO:tensorflow: (26.799 sec)
INFO:tensorflow:loss = 0.337652, step = 4702 (60.131 sec)
INFO:tensorflow: (26.673 sec)
INFO:tensorflow:global_step/sec: 1.68828
INFO:tensorflow: (32.559 sec)
INFO:tensorflow:loss = 0.207093, step = 4802 (59.233 sec)
INFO:tensorflow: (31.272 sec)
INFO:tensorflow:global_step/sec: 1.73412
INFO:tensorflow: (26.456 sec)
INFO:tensorflow:loss = 0.288307, step = 4902 (57.728 sec)
INFO:tensorflow: (31.555 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 1.89842
INFO:tensorflow: (26.832 sec)
INFO:tensorflow:loss = 0.152057, step = 9302 (52.673 sec)
INFO:tensorflow: (28.466 sec)
INFO:tensorflow:global_step/sec: 1.64813
INFO:tensorflow: (32.211 sec)
INFO:tensorflow:loss = 0.158741, step = 9402 (60.677 sec)
INFO:tensorflow: (27.567 sec)
INFO:tensorflow:global_step/sec: 1.87058
INFO:tensorflow: (25.962 sec)
INFO:tensorflow:loss = 0.266752, step = 9502 (53.534 sec)
INFO:tensorflow: (31.743 sec)
INFO:tensorflow:global_step/sec: 1.55876
INFO:tensorflow: (32.339 sec)
INFO:tensorflow:loss = 0.119461, step = 9602 (64.077 sec)
INFO:tensorflow: (26.666 sec)
INFO:tensorflow:global_step/sec: 1.90998
INFO:tensorflow: (25.690 sec)
INFO:tensorflow:loss = 0.232799, step = 9702 (52.357 sec)
INFO:tensorflow: (26.670 sec)
INFO:tensorflow:global_step/sec: 1.69278
INFO:tensorflow: (32.465 sec)
INFO:tensorflow:loss = 0.21181, step = 9802 (59.136 sec)
INFO:tensorflow: (28.301 sec)
INFO:tensorflow:global_step/sec: 1.83944
INFO:t

INFO:tensorflow: (29.344 sec)
INFO:tensorflow:loss = 0.177556, step = 14202 (61.745 sec)
INFO:tensorflow: (25.996 sec)
INFO:tensorflow:global_step/sec: 1.91394
INFO:tensorflow: (26.319 sec)
INFO:tensorflow:loss = 0.181376, step = 14302 (52.316 sec)
INFO:tensorflow: (29.188 sec)
INFO:tensorflow:global_step/sec: 1.62625
INFO:tensorflow: (32.238 sec)
INFO:tensorflow:loss = 0.22964, step = 14402 (61.424 sec)
INFO:tensorflow: (26.023 sec)
INFO:tensorflow:Saving checkpoints for 14495 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:global_step/sec: 1.89543
INFO:tensorflow: (26.735 sec)
INFO:tensorflow:loss = 0.144208, step = 14502 (52.759 sec)
INFO:tensorflow: (29.417 sec)
INFO:tensorflow:global_step/sec: 1.61884
INFO:tensorflow: (32.412 sec)
INFO:tensorflow:loss = 0.192915, step = 14602 (61.829 sec)
INFO:tensorflow: (27.053 sec)
INFO:tensorflow:global_step/sec: 1.88882
INFO:tensorflow: (25.834 sec)
INFO:tensorflow:loss = 0.101436, step = 14702 (52.886 sec)
INFO:tensorflow: (30.963 

INFO:tensorflow: (29.373 sec)
INFO:tensorflow:global_step/sec: 1.61558
INFO:tensorflow: (32.588 sec)
INFO:tensorflow:loss = 0.0696901, step = 19102 (61.962 sec)
INFO:tensorflow: (26.157 sec)
INFO:tensorflow:global_step/sec: 1.91342
INFO:tensorflow: (26.041 sec)
INFO:tensorflow:loss = 0.180504, step = 19202 (52.200 sec)
INFO:tensorflow: (30.818 sec)
INFO:tensorflow:global_step/sec: 1.59557
INFO:tensorflow: (31.854 sec)
INFO:tensorflow:loss = 0.159419, step = 19302 (62.671 sec)
INFO:tensorflow: (26.223 sec)
INFO:tensorflow:global_step/sec: 1.90375
INFO:tensorflow: (26.361 sec)
INFO:tensorflow:loss = 0.119578, step = 19402 (52.585 sec)
INFO:tensorflow: (31.722 sec)
INFO:tensorflow:global_step/sec: 1.55301
INFO:tensorflow: (32.615 sec)
INFO:tensorflow:loss = 0.0320463, step = 19502 (64.335 sec)
INFO:tensorflow: (26.790 sec)
INFO:tensorflow:global_step/sec: 1.88812
INFO:tensorflow: (26.171 sec)
INFO:tensorflow:loss = 0.0794766, step = 19602 (52.961 sec)
INFO:tensorflow: (29.302 sec)
INFO:te

SystemExit: 

e:\python\anaconda\anaconda\envs\cnn\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
